# Introduction

In this notebook there are few analysis on the [Chicago Taxi Trips](https://console.cloud.google.com/bigquery?p=bigquery-public-data&d=chicago_taxi_trips&page=dataset&inv=1&invt=Ab3rgg&project=grand-sphere-466709-g1&ws=!1m9!1m4!4m3!1sbigquery-public-data!2schicago_taxi_trips!3staxi_trips!1m3!3m2!1sbigquery-public-data!2schicago_taxi_trips) dataset from GoogleCloud


In [ ]:
!pip install gdown
!pip install geodatasets

import gdown
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame
import geodatasets
print("Setup Complete")

The following code cell fetches the `taxi_trips` table from the `chicago_taxi_trips` dataset.

In [ ]:
# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "chicago_taxi_trips" dataset
dataset_ref = client.dataset("chicago_taxi_trips", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# To find the table name
tables = list(client.list_tables(dataset))

# Print names of all tables in the dataset (there's only one!)
for table in tables:  
    print(table.table_id)

# Construct a reference to the "taxi_trips" table
table_ref = dataset_ref.table("taxi_trips")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

### 1) Number of trips in each year

In [ ]:
rides_per_year_query = """SELECT COUNT(1) AS num_trips, EXTRACT(YEAR FROM trip_start_timestamp) as year
                          FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips` 
                          GROUP BY year
                          ORDER BY year DESC 
                            """

# Set up the query (cancel the query if it would use too much of 
# your quota)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
rides_per_year_query_job = client.query(rides_per_year_query, job_config = safe_config) # Your code goes here

# API request - run the query, and return a pandas DataFrame
rides_per_year_result = rides_per_year_query_job.to_dataframe() # Your code goes here

# View results
print(rides_per_year_result)

### 2) Number of trips in each month

In [ ]:
rides_per_month_query = """SELECT EXTRACT(MONTH FROM trip_start_timestamp) as month, COUNT(1) AS num_trips
                          FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips` 
                          WHERE EXTRACT(YEAR FROM trip_start_timestamp) = 2016
                          GROUP BY month 
                          ORDER BY month  
                            """

# Set up the query
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
rides_per_month_query_job = client.query(rides_per_month_query, job_config = safe_config) # Your code goes here

# API request - run the query, and return a pandas DataFrame
rides_per_month_result = rides_per_month_query_job.to_dataframe() # Your code goes here

# View results
print(rides_per_month_result)

### 3) Average Speed

The query shows, for each hour of the day in the dataset, the corresponding number of trips and average speed.

The result will have three columns:
- `hour_of_day` - holds the result of extracting the hour from `trip_start_timestamp`.
- `num_trips` - the count of the total number of trips in each hour of the day (e.g. how many trips were started between 6AM and 7AM, independent of which day it occurred on).
- `avg_mph` - the average speed, measured in miles per hour, for trips that started in that hour of the day.  Average speed in miles per hour is calculated as `3600 * SUM(trip_miles) / SUM(trip_seconds)`. (The value 3600 is used to convert from seconds to hours.)

The query is restricted to data meeting the following criteria:
- a `trip_start_timestamp` > **2016-01-01** and < **2016-04-01**
- `trip_seconds` > 0 and `trip_miles` > 0


In [ ]:
speeds_query = """
               WITH RelevantRides AS
               (
                   SELECT EXTRACT(HOUR FROM trip_start_timestamp) AS hour_of_day, trip_miles, trip_seconds
                   FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips` 
                   WHERE trip_seconds > 0 AND trip_miles > 0
                   AND trip_start_timestamp > '2016-01-01' AND
                   trip_start_timestamp < '2016-04-01'
               )
               SELECT hour_of_day,
               COUNT(1) AS num_trips, 
               3600*SUM(trip_miles) / SUM(trip_seconds) AS avg_mph
               FROM RelevantRides
               GROUP BY hour_of_day
               ORDER BY hour_of_day
               """

# Set up the query
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
speeds_query_job = client.query(speeds_query, job_config = safe_config) # Your code here

# API request - run the query, and return a pandas DataFrame
speeds_result = speeds_query_job.to_dataframe() # Your code here

# View results
print(speeds_result)

### 4) Predicting the taxis demand

The final dataset will have two columns:
- `trip_date` - contains one entry for each date from January 1, 2016, to March 31, 2016.
- `avg_num_trips` - shows the average number of daily trips, calculated over a window including the value for the current date, along with the values for the preceding 3 days and the following 3 days, as long as the days fit within the three-month time frame. 
For instance, when calculating the value in this column for January 3, 2016, the window will include the number of trips for the preceding 2 days, the current date, and the following 3 days.

In [ ]:
avg_num_trips_query = """
                      WITH trips_by_day AS
                      (
                      SELECT DATE(trip_start_timestamp) AS trip_date,
                          COUNT(*) as num_trips
                      FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                      WHERE trip_start_timestamp > '2016-01-01' AND trip_start_timestamp < '2016-04-01'
                      GROUP BY trip_date
                      ORDER BY trip_date
                      )
                      SELECT trip_date,
                          AVG(num_trips) OVER (
                               ORDER BY trip_date
                               ROWS BETWEEN 3 PRECEDING AND 3 FOLLOWING
                               ) AS avg_num_trips
                      FROM trips_by_day
                      """

# Set up the query
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
avg_num_query_job = client.query(avg_num_trips_query, job_config = safe_config) # Your code here

# API request - run the query, and return a pandas DataFrame
avg_num_result = avg_num_query_job.to_dataframe() # Your code here

# View results
print(avg_num_result)

### 5) Separate and order trips by community area

The returned additional column called `trip_number` will shows the order in which the trips were taken from their respective community areas.
So, the first trip of the day originating from community area 1 should receive a value of 1; the second trip of the day from the same area should receive a value of 2.  Likewise, the first trip of the day from community area 2 should receive a value of 1, and so on.

In [ ]:
trip_number_query = """
                    SELECT pickup_community_area,
                        trip_start_timestamp,
                        trip_end_timestamp,
                        RANK()
                          OVER(
                           PARTITION BY pickup_community_area
                           ORDER BY trip_start_timestamp  
                        ) AS trip_number
                    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                    WHERE DATE(trip_start_timestamp) = '2013-10-03'
                    """
# Set up the query
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
trip_num_query_job = client.query(trip_number_query, job_config = safe_config) # Your code here

# API request - run the query, and return a pandas DataFrame
trip_num_result = trip_num_query_job.to_dataframe() # Your code here

# View results
print(trip_num_result)

: 

### 6) How much time elapses between trips?

Length of the break (in minutes) that the driver had before each trip started (this corresponds to the time between `trip_start_timestamp` of the current trip and `trip_end_timestamp` of the previous trip).

In [ ]:
break_time_query = """
                   SELECT taxi_id,
                       trip_start_timestamp,
                       trip_end_timestamp,
                       TIMESTAMP_DIFF(
                           trip_start_timestamp, 
                           LAG(trip_end_timestamp) 
                               OVER (
                                    PARTITION BY taxi_id 
                                    ORDER BY trip_start_timestamp), 
                           MINUTE) as prev_break
                   FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                   WHERE DATE(trip_start_timestamp) = '2013-10-03' 
                   """

# Set up the query
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
break_time_query_job = client.query(break_time_query, job_config = safe_config) # Your code here

# API request - run the query, and return a pandas DataFrame
break_time_result = break_time_query_job.to_dataframe() # Your code here

# View results
print(break_time_result)

### 7) Passenger Load Factor (Efficiency Ratio)?

Length of the break (in minutes) that the driver had before each trip started (this corresponds to the time between `trip_start_timestamp` of the current trip and `trip_end_timestamp` of the previous trip).


How much of a taxi's working day is spent transporting passengers?
Goal: Measure taxi efficiency based on active time on the road vs idle time. Need to select the cumulative time during which the taxi driver worked. 
The load factor tells how many active hours wrt the total shift hour

In [ ]:
break_time_query = """ SELECT taxi_id,
DATE(trip_start_timestamp) AS trip_date,
TIMESTAMP_DIFF(MAX(trip_end_timestamp), MIN(trip_start_timestamp), MINUTE )/60 as total_shift_hour,
SAFE_DIVIDE(SUM(trip_seconds)/3600.0, TIMESTAMP_DIFF(MAX(trip_end_timestamp), MIN(trip_start_timestamp), MINUTE )/60) AS load_factor
FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
WHERE trip_start_timestamp BETWEEN '2016-01-01' AND '2016-01-31'
GROUP BY taxi_id, trip_date
LIMIT 5000 """

# Set up the query
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
break_time_query_job = client.query(break_time_query, job_config = safe_config) # Your code here

# API request - run the query, and return a pandas DataFrame
break_time_result = break_time_query_job.to_dataframe() # Your code here

# View results
print(break_time_result)

### 8) Peak Demand Window Detection (15-min intervals)

Goal: Segment the 24 hours into 15-minute intervals to identify peaks in demand (e.g. rush hour 7:15–7:30).

In [ ]:
peak_demand_query = """ SELECT FORMAT_TIMESTAMP('%H:%M', TIMESTAMP_TRUNC(trip_start_timestamp, MINUTE)) AS time_bucket, 
COUNT(*) AS num_trips 
FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
WHERE trip_start_timestamp BETWEEN '2016-01-01' AND '2016-03-31'
GROUP BY time_bucket
ORDER BY time_bucket ASC
LIMIT 5000  """

# TIMESTAMP_TRUNC: Truncates a TIMESTAMP or DATETIME value at a particular granularity.
# FORMAT_TIMESTAMP: Formats a TIMESTAMP value according to the specified format string.

# Set up the query
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
peak_demand_query_job = client.query(peak_demand_query, job_config = safe_config) # Your code here

# API request - run the query, and return a pandas DataFrame
peak_demand_result = peak_demand_query_job.to_dataframe() # Your code here

# View results
print(peak_demand_result)

: 

### 9) Pickup Latitude and Longitude of the pickup location (to be printed later on the map)

In [ ]:
lat_lon_query = '''  SELECT
  pickup_latitude,
  pickup_longitude, 
  
FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
WHERE pickup_latitude IS NOT NULL
  AND pickup_longitude IS NOT NULL
  --AND pickup_latitude BETWEEN 41.6 AND 42.1
  --AND pickup_longitude BETWEEN -87.95 AND -87.5
  AND EXTRACT(YEAR FROM trip_start_timestamp) = 2019
LIMIT 50000
'''

# Set up the query
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
lat_lon_query_job = client.query(lat_lon_query, job_config = safe_config) # Your code here

# API request - run the query, and return a pandas DataFrame
lat_lon_result = lat_lon_query_job.to_dataframe() # Your code here

# View results
print(lat_lon_result)

### 10) Statistics 

In [ ]:
stat_query = '''  SELECT AVG(fare) AS avg_fare,
AVG(tips) AS avg_tips,
AVG(tolls) AS avg_tolls,
AVG(extras) AS avg_extras, 
COUNT(*) AS num_trips,
AVG(trip_seconds)/60 AS avg_minutes,
EXTRACT (YEAR FROM trip_start_timestamp) AS year
FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips` 
WHERE trip_seconds > 0
GROUP BY year
ORDER BY year ASC
'''

# Set up the query
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
stat_query_job = client.query(stat_query, job_config = safe_config) # Your code here

# API request - run the query, and return a pandas DataFrame
stat_result = stat_query_job.to_dataframe() # Your code here

# View results
print(stat_result)

### 11) Time Between Dropoff and Next Pickup (Driver Idle Time)

In [ ]:
time_do_query = ''' 
SELECT taxi_id,
trip_start_timestamp,
trip_end_timestamp,
TIMESTAMP_DIFF( trip_start_timestamp,
LAG(trip_end_timestamp) -- Lag() function to access previous rows data as per defined offset value.
OVER ( PARTITION BY taxi_id ORDER BY trip_start_timestamp), MINUTE) AS idle_minutes
FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
WHERE EXTRACT(YEAR FROM trip_start_timestamp) = 2016
  AND trip_start_timestamp IS NOT NULL
  AND trip_end_timestamp IS NOT NULL
LIMIT 5000
'''

# Set up the query
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
time_do_query_job = client.query(time_do_query, job_config = safe_config) # Your code here

# API request - run the query, and return a pandas DataFrame
time_do_result = time_do_query_job.to_dataframe() # Your code here

# View results
print(time_do_query)

### 12) Number of trips during year/day/month

In [ ]:

trips_query = ''' 
SELECT COUNT(*) as num_trips,
EXTRACT(YEAR FROM trip_start_timestamp) AS year,
EXTRACT(MONTH FROM trip_start_timestamp) AS month,
EXTRACT(DAY FROM trip_start_timestamp) AS day
FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips` 
GROUP BY year, month, day
ORDER BY year DESC
'''

# Set up the query
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
trips_query_job = client.query(trips_query, job_config = safe_config) # Your code here

# API request - run the query, and return a pandas DataFrame
trips_result = trips_query_job.to_dataframe() # Your code here

# View results
print(trips_result)

# Plots associated with the queries above


In [ ]:
# Define the Google Drive public link
file_id = '1V4Q99BIsm2jKfyLs9ISWONHR9xthdjSN'
output_path = 'downloaded_file.csv'

#File path: https://drive.google.com/file/d/1V4Q99BIsm2jKfyLs9ISWONHR9xthdjSN/view?usp=sharing

# Download the file from Google Drive
try:
    gdown.download(f'https://drive.google.com/uc?id={file_id}', output_path, fuzzy=True)
    print(f"File downloaded successfully to {output_path}")

    # Read the downloaded CSV file into a pandas DataFrame
    df = pd.read_csv(output_path)
    print("CSV file read successfully!")

except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
# Create a bar plot of num_trips per year
plt.figure(figsize=(10, 6))
sns.barplot(x='year', y='num_trips', data=df)
plt.title('Number of Trips per Year')
plt.xlabel('Year')
plt.ylabel('Number of Trips')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Melt the DataFrame to long format for plotting multiple columns
df_melted = df.melt(id_vars=['year'],
                    value_vars=['avg_fare', 'avg_tips', 'avg_tolls', 'avg_extras'],
                    var_name='statistic',
                    value_name='value')

# Create a grouped bar plot
plt.figure(figsize=(12, 7))
sns.barplot(x='year', y='value', hue='statistic', data=df_melted, palette='viridis')
plt.title('Statistics over Years')
plt.xlabel('Year')
plt.ylabel('Value (A.U.)')
plt.xticks(rotation=45)
plt.legend(title='Statistic')
plt.tight_layout()
plt.show()

In [ ]:
gdown.download(f'https://drive.google.com/file/d/1wCy1ZdzBaWUDIDoHUqEjBEB9dmKVwCM3/view?usp=sharing', 'downloaded_lat_long.csv', fuzzy=True)

df_lat = pd.read_csv('downloaded_lat_long.csv', delimiter=',', skiprows=0, low_memory=False)
geometry = [Point(xy) for xy in zip(df_lat['pickup_longitude'], df_lat['pickup_latitude'])]
gdf = GeoDataFrame(df_lat, geometry=geometry)


#this is a simple map that goes with geopandas
# deprecated: world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world = gpd.read_file(geodatasets.data.naturalearth.land['url'])

base = world.plot(figsize=(10, 6))

# Aggiungi i punti (gdf) sopra la mappa
gdf.plot(ax=base, marker='o', color='red', markersize=15)

# Imposta i limiti sull'asse X e Y
base.set_xlim([-88, -87.5])  # Limiti asse X (longitudine) -- -87.95 AND -87.5
base.set_ylim([41.5, 42.1])   # Limiti asse Y (latitudine) -- 41.6 AND 42.1

plt.show()


In [ ]:
gdown.download(f'https://drive.google.com/file/d/1VEfYWNioWnO7cBkKHS297OJJToUtz9z1/view?usp=sharing', 'idle_minutes.csv', fuzzy=True)

df_idle_minutes = pd.read_csv('idle_minutes.csv', delimiter=',', skiprows=0, low_memory=False)
df_idle_minutes['idle_minutes'].plot(kind='hist', bins=20, title='idle_minutes')
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:

gdown.download(f'https://drive.google.com/file/d/1wC6gCgBeK0c_Oj-8S9Hd8nDFKpPZ4xPL/view?usp=sharing', 'peak_time_trips.csv', fuzzy=True)

df_peak_time_trips = pd.read_csv('peak_time_trips.csv', delimiter=',', skiprows=0, low_memory=False)
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['time_bucket']
  ys = series['num_trips']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = df_peak_time_trips.sort_values('time_bucket', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('time_bucket')
_ = plt.ylabel('num_trips')

In [ ]:
gdown.download(f'https://drive.google.com/file/d/1sAzs0fJiI113RkQJSvfapsX7FYSP0T_N/view?usp=sharing', 'active_time.csv', fuzzy=True)

df_active_time = pd.read_csv('active_time.csv', delimiter=',', skiprows=0, low_memory=False)
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['trip_date']
  ys = series['load_factor']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = df_active_time.sort_values('trip_date', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('trip_date')
_ = plt.ylabel('load_factor')